## Supervised Learning using BERT for Sentiment Analysis and Notation Prediction

### Data Extraction and Preprocessing (we follow the same process as in data_cleaning but we keep a different sample of the data)

In [45]:
import pandas as pd
import nltk
from cleantext import clean
import tqdm
import numpy as np
from deep_translator import GoogleTranslator

In [46]:
reviews_df = pd.read_csv("data/reviews.csv", sep=";")
reviews_df.head(3)

,category_link,company_link,company_name,score,description,review_page_nb,review_link,review_score,review_title,review_text,category
0,https://fr.trustpilot.com/categories/food_beverages_tobacco?verified=true,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9","Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet",329,https://fr.trustpilot.com/reviews/65a5388a60d6a4425263fe42,5,Application conviviale pour passer ses…,Application conviviale pour passer ses commandes. Manquent juste les ingrédients pas évidents à trouver.Notification par SMS peu de temps avant le créneau choisi pour un créneau précis de 20 minutes.Livreurs agréables.Merci le Fourgon !,food_beverages_tobacco
1,https://fr.trustpilot.com/categories/food_beverages_tobacco?verified=true,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9","Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet",329,https://fr.trustpilot.com/reviews/65a53245a223c5f420a9cee3,5,Très facile pour la commande,"Très facile pour la commande, très rapide et livraison très professionnelle! Le livreur a apporté les bacs chez nous car ils étaient lourds! 12 bouteilles par bacs... soit 12kg sans compter le poids des bouteilles. Très pro et sympa!Vraiment chouette et écologique! On recommandera sans hésiter! Fini les microparticules de plastique!",food_beverages_tobacco
2,https://fr.trustpilot.com/categories/food_beverages_tobacco?verified=true,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9","Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet",329,https://fr.trustpilot.com/reviews/659f0e15dd570fb981a1d433,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir trouvé un système de consigne pour un choix plus large de produits qu'en magasin.Créneau horaire de livraison respecté, avec une information précise sur l'approche du livreur.Livreur sympathique, souriant et serviable.À bientôt pour une nouvelle commande !",food_beverages_tobacco


In [47]:
reviews_df = reviews_df[["company_name", "score", "review_score", "review_title", "review_text", "category"]]

In [48]:
# we merge the title and the text of the review
reviews_df["review"] = reviews_df["review_title"] + " " + reviews_df["review_text"]
reviews_df.head(3)

,company_name,score,review_score,review_title,review_text,category,review
0,Le Fourgon,"4,9",5,Application conviviale pour passer ses…,Application conviviale pour passer ses commandes. Manquent juste les ingrédients pas évidents à trouver.Notification par SMS peu de temps avant le créneau choisi pour un créneau précis de 20 minutes.Livreurs agréables.Merci le Fourgon !,food_beverages_tobacco,Application conviviale pour passer ses… Application conviviale pour passer ses commandes. Manquent juste les ingrédients pas évidents à trouver.Notification par SMS peu de temps avant le créneau choisi pour un créneau précis de 20 minutes.Livreurs agréables.Merci le Fourgon !
1,Le Fourgon,"4,9",5,Très facile pour la commande,"Très facile pour la commande, très rapide et livraison très professionnelle! Le livreur a apporté les bacs chez nous car ils étaient lourds! 12 bouteilles par bacs... soit 12kg sans compter le poids des bouteilles. Très pro et sympa!Vraiment chouette et écologique! On recommandera sans hésiter! Fini les microparticules de plastique!",food_beverages_tobacco,"Très facile pour la commande Très facile pour la commande, très rapide et livraison très professionnelle! Le livreur a apporté les bacs chez nous car ils étaient lourds! 12 bouteilles par bacs... soit 12kg sans compter le poids des bouteilles. Très pro et sympa!Vraiment chouette et écologique! On recommandera sans hésiter! Fini les microparticules de plastique!"
2,Le Fourgon,"4,9",5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir trouvé un système de consigne pour un choix plus large de produits qu'en magasin.Créneau horaire de livraison respecté, avec une information précise sur l'approche du livreur.Livreur sympathique, souriant et serviable.À bientôt pour une nouvelle commande !",food_beverages_tobacco,"Première expérience réussie ! Pour nous, c'était une première.Ravis d'avoir trouvé un système de consigne pour un choix plus large de produits qu'en magasin.Créneau horaire de livraison respecté, avec une information précise sur l'approche du livreur.Livreur sympathique, souriant et serviable.À bientôt pour une nouvelle commande !"


In [49]:
clean(reviews_df["review"].values[0], no_emoji=True, no_line_breaks=True, lower=False).replace("#", "")

'Application conviviale pour passer ses Application conviviale pour passer ses commandes. Manquent juste les ingredients pas evidents a trouver.Notification par SMS peu de temps avant le creneau choisi pour un creneau precis de 20 minutes.Livreurs agreables.Merci le Fourgon !'

In [50]:
reviews_df["review"] = reviews_df["review"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

In [51]:
reviews_df["review"].values[100]

"Exceptionnel La brulerie Belleville est devenue mon unique fournisseur en cafe, le cafe est vraiment d'une qualite exceptionnelle ! De plus, le service apres-vente est d'une qualite humaine devenue bien rare a present !"

#### We Keep only a sample of the data (the top 100 companies with the most reviews) Because of the traduction cost of the "review" column

In [58]:
# number of reviews per company
reviews_df["company_name"].value_counts()[:10]

company_name
Hardloop          236
Alltricks         210
Ekosport          140
vertbaudet        140
Mode Tactique     120
Weenect           120
AAAEP             120
Handball Store    111
Meyclub           100
Bleen             100
Name: count, dtype: int64

In [60]:
# number of reviews in all the dataset
len(reviews_df)

235503

In [53]:
# print the first company name
reviews_df["company_name"].value_counts().index[0]

'Hardloop\xa0'

In [54]:
reviews_df["company_name"] = reviews_df["company_name"].apply(lambda x: x.replace("\xa0", "") if x != "\xa0" else x)
reviews_df["company_name"].value_counts().index[0]

'Hardloop'

In [62]:
# we keep only the reviews of the first 100 companies in terms of number of reviews
revsample_df = reviews_df[reviews_df["company_name"].isin(reviews_df["company_name"].value_counts()[:100].index)]
revsample_df["company_name"].value_counts().sum()

8972

In [56]:
revsample_df["company_name"].value_counts()[:10]

company_name
Hardloop          236
Alltricks         210
vertbaudet        140
Ekosport          140
Mode Tactique     120
Weenect           120
AAAEP             120
Handball Store    111
Bleen             100
Meyclub           100
Name: count, dtype: int64

In [65]:
revsample_df = revsample_df.dropna(ignore_index=True)

In [67]:
# we display all the lines of the first company
pd.set_option("display.max_colwidth", None)
revsample_df[revsample_df["company_name"] == "Hardloop"]

,company_name,score,review_score,review_title,review_text,category,review
7658,Hardloop,"4,5",1,Première commande sur ce site,"Première commande sur ce site, et extrêmement déçue !Pantalon de randonnée commandé pour un cadeau de Noël début décembre, il n’est toujours pas réceptionné malgré les dates indiquées sur le site. Diverses relances auprès du service client par e-mail et Instagram afin d’obtenir à minima un retour sur la livraison ou sur un remboursement éventuel pour l’acheter ailleurs : aucun retour.Une honte.À fuir…Merci de supprimer mon adresse e-mail privée à ce commentaire (méthode très douteuse pour que je supprime mon commentaire négatif).",sports,"Premiere commande sur ce site Premiere commande sur ce site, et extremement decue !Pantalon de randonnee commande pour un cadeau de Noel debut decembre, il n'est toujours pas receptionne malgre les dates indiquees sur le site. Diverses relances aupres du service client par e-mail et Instagram afin d'obtenir a minima un retour sur la livraison ou sur un remboursement eventuel pour l'acheter ailleurs : aucun retour.Une honte.A fuirMerci de supprimer mon adresse e-mail privee a ce commentaire (methode tres douteuse pour que je supprime mon commentaire negatif)."
7659,Hardloop,"4,5",1,Pantalon à 150€ reçu 10 jours après la…,"Pantalon à 150€ reçu 10 jours après la commande. Pour cause de taille trop grande, impossible d'obtenir un bon de retour pour retourner l'article, car le site émet systématiquement un message d'erreur quand on en fait la demande. Le numéro de tel du prétendu standard n'est en fait qu'un simple répondeur, où il n'est même pas possible de laisser un message.Finalement, après deux mails envoyés, j'ai fini par recevoir un bon de retour imprimable. Pas de remboursement reçu un mois après l'envoi... mais reçu immédiatement après après que j'aie renvoyé un mailde reclamation. À fuir.",sports,"Pantalon a 150 recu 10 jours apres la Pantalon a 150 recu 10 jours apres la commande. Pour cause de taille trop grande, impossible d'obtenir un bon de retour pour retourner l'article, car le site emet systematiquement un message d'erreur quand on en fait la demande. Le numero de tel du pretendu standard n'est en fait qu'un simple repondeur, ou il n'est meme pas possible de laisser un message.Finalement, apres deux mails envoyes, j'ai fini par recevoir un bon de retour imprimable. Pas de remboursement recu un mois apres l'envoi... mais recu immediatement apres apres que j'aie renvoye un mailde reclamation. A fuir."
7660,Hardloop,"4,5",5,Equipe très réactive en cas de problème…,Equipe très réactive en cas de problème sur une commande pour trouver des solutions.,sports,Equipe tres reactive en cas de probleme Equipe tres reactive en cas de probleme sur une commande pour trouver des solutions.
7661,Hardloop,"4,5",4,Très bon service,"Beaucoup de choix, produits bien emballés, et livraison rapide.",sports,"Tres bon service Beaucoup de choix, produits bien emballes, et livraison rapide."
7662,Hardloop,"4,5",5,Parfait,"Parfait, livraison rapide et service optimal, je recommande",sports,"Parfait Parfait, livraison rapide et service optimal, je recommande"
...,...,...,...,...,...,...,...
8676,Hardloop,"4,0",5,"Competitive price, rapid delivery","Competitive price, rapid delivery to Poland",vehicles_transportation,"Competitive price, rapid delivery Competitive price, rapid delivery to Poland"
8677,Hardloop,"4,0",1,Szczytem chamstwa nie jest dostarczyć…,"Szczytem chamstwa nie jest dostarczyć produkt, który zgodnie z prawem zakupiłem, i który jest dostępny w sklepie, tylko dlatego, że właścicielowi nie podoba się kwota, za którą został sprzedany. W systemie była widoczna cena w promocji. I z niej korzystając kupiłem kurtkę Helly Hannsena. Anulowanie mojego kupna i zwrot pieniędzy zasłaniając się rzekomym błędem w wyświetlaniu ceny jest co najmniej nie na miejscu. Jeżeli nawet do takiego doszło właściciel powinien rozwiązywać tą kwestię, wśród swoich pracowników odpowiedzialnych za to

We keep only the reviews in french

In [71]:
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [72]:
def detecte_langage(message):
    # on definie un dictionnaire vide
    # {langue : nbre de stopwords communs 
    # entre langue et les mots de message}
    languages_shared_words = {}
    # tokenization en mots
    words = wordpunct_tokenize(message)
    for language in stopwords.fileids():
        # stopwords pour chaque langue
        stopwords_liste = stopwords.words(language)
        # on retire les doublons
        words = set(words)
        # les mots communs entre stopwords 
        # d'une langue et les mots de message
        common_elements = words.intersection(stopwords_liste)
        # ajout du couple au dictionnaire
        languages_shared_words[language] = len(common_elements)
    # on retourne la langue avec le max de mots commun
    return  max(languages_shared_words, key = languages_shared_words.get)

In [73]:
# on applique la fonction pour chaque message sur la colonne review et on ne garde que les lignes ou la langue est en francais
revsample_df["langue"] = revsample_df["review"].apply(detecte_langage)

In [74]:
revsample_df[revsample_df["langue"] == "french"].count()

company_name    7729
score           7729
review_score    7729
review_title    7729
review_text     7729
category        7729
review          7729
langue          7729
dtype: int64

In [75]:
revsample_df_fr = revsample_df[revsample_df["langue"] == "french"]

Training sample : 

In [1]:
from transformers import CamembertTokenizer

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


ImportError: cannot import name 'CamembertTokenizer' from 'transformers' (c:\Users\gdetr\anaconda3\envs\global\Lib\site-packages\transformers\__init__.py)

In [1]:
print("balvke")

balvke


In [3]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [8]:
! pip install torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121

In [9]:
import torch

ModuleNotFoundError: No module named 'torch'

In [4]:
torch.cuda.is_available()

NameError: name 'torch' is not defined